In [1]:
import pandas as pd 
import numpy as np

from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [4]:
import os
for dirname, _, filenames in os.walk('C:\Eric\M5_Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\M5_Data\calendar.csv
C:\Eric\M5_Data\disaster_minji.csv
C:\Eric\M5_Data\sales_train_evaluation.csv
C:\Eric\M5_Data\sales_train_validation.csv
C:\Eric\M5_Data\sample_submission.csv
C:\Eric\M5_Data\sell_prices.csv
C:\Eric\M5_Data\SNAP_CA.csv
C:\Eric\M5_Data\SNAP_TX.csv
C:\Eric\M5_Data\SNAP_WI.csv


In [6]:
SNAP_pl_ca = pd.read_csv("C:\Eric\M5_Data\SNAP_CA.csv")
SNAP_pl_tx = pd.read_csv("C:\Eric\M5_Data\SNAP_TX.csv")
SNAP_pl_wi = pd.read_csv("C:\Eric\M5_Data\SNAP_WI.csv")

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

#
# reduce_mem_usage(SNAP_pl_ca)
# reduce_mem_usage(SNAP_pl_tx)
# reduce_mem_usage(SNAP_pl_wi)


In [11]:
SNAP_pl_ca.columns = ['date', 'recipient']
SNAP_pl_wi.columns = ['date', 'recipient']
SNAP_pl_tx.columns = ['date', 'recipient']

In [12]:
SNAP_pl_ca['date'] = pd.to_datetime(SNAP_pl_ca.date, errors='coerce')
SNAP_pl_wi['date'] = pd.to_datetime(SNAP_pl_wi.date, errors='coerce')
SNAP_pl_tx['date'] = pd.to_datetime(SNAP_pl_tx.date, errors='coerce')


In [13]:
SNAP_pl_ca['date_month'] = SNAP_pl_ca['date'].dt.strftime('%m')
SNAP_pl_wi['date_month'] = SNAP_pl_wi['date'].dt.strftime('%m')
SNAP_pl_tx['date_month'] = SNAP_pl_tx['date'].dt.strftime('%m')

In [14]:
SNAP_pl_ca['date_year'] = SNAP_pl_ca['date'].dt.strftime('%Y')
SNAP_pl_wi['date_year'] = SNAP_pl_wi['date'].dt.strftime('%Y')
SNAP_pl_tx['date_year'] = SNAP_pl_tx['date'].dt.strftime('%Y')

In [15]:

# CA type change 
SNAP_pl_ca.date_month = pd.to_numeric(SNAP_pl_ca.date_month)
SNAP_pl_ca.date_year = pd.to_numeric(SNAP_pl_ca.date_year)
# WI type change 
SNAP_pl_ca.date_month = pd.to_numeric(SNAP_pl_wi.date_month)
SNAP_pl_ca.date_year = pd.to_numeric(SNAP_pl_wi.date_year)
# TX type change 
SNAP_pl_ca.date_month = pd.to_numeric(SNAP_pl_tx.date_month)
SNAP_pl_ca.date_year = pd.to_numeric(SNAP_pl_tx.date_year)

In [16]:
SNAP_pl_ca = SNAP_pl_ca.rename(columns = {'date_month':'month'} )
SNAP_pl_wi = SNAP_pl_wi.rename(columns = {'date_month':'month'} )
SNAP_pl_tx = SNAP_pl_tx.rename(columns = {'date_month':'month'} )

SNAP_pl_ca = SNAP_pl_ca.rename(columns = {'date_year':'year'} )
SNAP_pl_wi = SNAP_pl_wi.rename(columns = {'date_year':'year'} )
SNAP_pl_tx = SNAP_pl_tx.rename(columns = {'date_year':'year'} )

In [20]:
SNAP_pl_wi.month = pd.to_numeric(SNAP_pl_wi.month)
SNAP_pl_wi.year = pd.to_numeric(SNAP_pl_wi.year)
SNAP_pl_tx.month = pd.to_numeric(SNAP_pl_tx.month)
SNAP_pl_tx.year = pd.to_numeric(SNAP_pl_tx.year)

In [21]:
calendar         = pd.read_csv("C:\Eric\M5_Data\calendar.csv")
train_validation = pd.read_csv("C:\Eric\M5_Data\sales_train_validation.csv")
sell_prices = pd.read_csv("C:\Eric\M5_Data\sell_prices.csv")

In [45]:
cal01 = calendar.copy()
cal01.shape

(1969, 14)

In [46]:
cal02 = calendar.copy()
cal02.shape

(1969, 14)

In [43]:
SNAP_pl_ca = SNAP_pl_ca.drop(['date'],axis=1)
SNAP_pl_ca.columns

Index(['recipient', 'month', 'year'], dtype='object')

In [44]:
SNAP_pl_tx = SNAP_pl_tx.drop(['date'],axis=1)
SNAP_pl_tx.columns
SNAP_pl_wi = SNAP_pl_wi.drop(['date'],axis=1)
SNAP_pl_wi.columns

Index(['recipient', 'month', 'year'], dtype='object')

In [ ]:
cal02

In [47]:
# SNAP_pl_ca 에 있는 recipient 숫자를 "year" & "month" 기준으로 cal01 데이터에 merge. 
mergex001 = pd.merge(cal02, SNAP_pl_ca, on =["month","year"], how="left"  )

In [48]:
mergex001.shape

(1969, 15)

In [49]:
mergex001 = mergex001.rename(columns = {'recipient':'ca_recipient'} )

In [50]:
mergex001 = pd.merge(mergex001,SNAP_pl_wi, on =["month","year"], how="left"  )

In [52]:
mergex001 = mergex001.rename(columns = {'recipient':'wi_recipient'} )

In [53]:
mergex001 = pd.merge(mergex001,SNAP_pl_tx,on =["month","year"], how="left" )

In [54]:
mergex001 = mergex001.rename(columns = {'recipient':'wx_recipient'} )

In [57]:
mergex001.columns

Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'ca_recipient', 'wi_recipient',
       'wx_recipient'],
      dtype='object')

In [58]:
mergex001.to_csv("calendar_add_snap.csv",index=False)